In [1]:
# modelo vgg16 usado para entrenar por transferencia las 10 categorías de CIFAR10
import sys
from matplotlib import pyplot
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
# definir el modelo cnn
def define_model():
    # cargar el modelo
    model = VGG16(include_top=False, input_shape=(224, 224, 3))
    # marcar las capas cargadas como no entrenables
    for layer in model.layers:
        layer.trainable = False
    # añadir nuevas capas del clasificador
    flat1 = Flatten()(model.layers[-1].output)
    class1 = Dense(128, activation='relu', kernel_initializer='he_uniform')(flat1)
    output = Dense(2, activation='softmax')(class1)
    # definir nuevo modelo
    model = Model(inputs=model.inputs, outputs=output)
    # compilar el modelo
    opt = SGD(learning_rate=0.001, momentum=0.9)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [3]:
# mostrar las curvas de diagnóstico del aprendizaje
def summarize_diagnostics(history):
    # plot error
    pyplot.subplot(211)
    pyplot.title('Cross Entropy Loss')
    pyplot.plot(history.history['loss'], color='blue', label='train')
    pyplot.plot(history.history['val_loss'], color='orange', label='test')
    # plot exactitud
    pyplot.subplot(212)
    pyplot.title('Classification Accuracy')
    pyplot.plot(history.history['accuracy'], color='blue', label='train')
    pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
    # guardar el plot en un archivo ... por si lo queremos publicar
    filename = sys.argv[0].split('/')[-1]
    pyplot.savefig(filename + '_plot.png')
    pyplot.close()

In [4]:
# ejecutamos el ambiente de evaluación de modelos
def run_amb_pruebas():
    # definir modelo
    model = define_model()
    # crear data generator
    datagen = ImageDataGenerator(featurewise_center=True)
    # especificar los valores promedio "mean" de imagenet para centrado
    datagen.mean = [123.68, 116.779, 103.939]
    # preparar iterador para las clases de imagenes en CIFAR10
    train_it = datagen.flow_from_directory('data/train/', class_mode='categorical', batch_size=64, target_size=(224, 224))
    test_it = datagen.flow_from_directory('data/test/', class_mode='categorical', batch_size=64, target_size=(224, 224))
    # entrenar el modelo
    history = model.fit(train_it, steps_per_epoch=len(train_it), validation_data=test_it, validation_steps=len(test_it), epochs=10, verbose=1)
    # evaluar el modelo
    _, acc = model.evaluate(test_it, steps=len(test_it), verbose=0)
    print('> %.3f' % (acc * 100.0))
    # curvas de de aprendizaje
    summarize_diagnostics(history)



In [5]:
# punto de entrada del ambiente de evaluación de modelos
run_amb_pruebas()

Found 9 images belonging to 2 classes.
Found 5 images belonging to 2 classes.


C:\Users\sebas\anaconda3\envs\Web\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.6667 - loss: 4.2354 - val_accuracy: 0.4000 - val_loss: 75.3216
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.3333 - loss: 54.1297 - val_accuracy: 0.6000 - val_loss: 30.0030
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 0.6000 - val_loss: 64.9951
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 0.6000 - val_loss: 98.5590
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.8889 - loss: 1.4431 - val_accuracy: 0.6000 - val_loss: 105.9619
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 0.6000 - val_loss: 111.3156
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 0.6000 - val_loss: 114.8137
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 0.6000 - va